# Age Influence and Information Spreading

In [4]:
import pandas as pd
from pandas import DataFrame

import io
import networkx as nx
#from google.colab import files

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
#from sympy import deg
%matplotlib inline
import random
import math


# Age Influence
### Factors: Older than 5 years, Older than 10 years, # of season played together

In [5]:
def age_influence(G,t_tot,init): 
    # G = graph; t_tot = number of timesteps to run the model for; init = list of nodes that are activated at t=0
    numOlder5 = 0
    numOlder10 = 0
    t = 0
    
    #avg_weight = statistics.mean([e[2]['Weight'] for e in G.edges(data=True)])
    #max_weight = max([e[2]['Weight'] for e in G.edges(data=True)])
    multiplier = 0.1
    
    activation_times = {}
    for i in init:
        activation_times[i]=0
    
    while t<t_tot:
    
        curr_infectious = [n for n in activation_times if activation_times[n]==t]

        for n in curr_infectious:
            for m in G.neighbors(n):
                if m not in activation_times.keys():
                    multiplier = 0.1
                    if(int(G.nodes[m]['DOB']) - int(G.nodes[n]['DOB']) > 9):
                        multiplier+= 0.5
                        numOlder10+=1
                    elif(int(G.nodes[m]['DOB']) - int(G.nodes[n]['DOB']) > 4):
                        multiplier+=0.2
                        numOlder5+=1
                    
                    p = G[n][m]['Weight']
                    if(p > 8):
                        multiplier+= 0.65
                    elif(p > 5):
                        multiplier+= 0.4
                    elif(p > 3):
                        multiplier+= 0.1
                    
                    # if p>random.uniform(0,1)*10:
                    #     activation_times[m] = t+1
                    if(multiplier >= 1):
                        
                        multiplier = 0.99
                    check = math.floor( random.uniform(0, 1/(1-multiplier)))
                    

                    if(check == 1):
                        activation_times[m] = t+1
                    
                        
                        
        t+=1
    print(numOlder10, numOlder5)
    return activation_times

# Information Cascade
### Factors: Same Nation, # of teams played on together, # of season played together

In [6]:
import random
import math

#More than 5 in weight -> Boost
#Same country -> Boost
#unique squads -> Boost


def information_cascade(G,t_tot,init): 
    # G = graph; t_tot = number of timesteps to run the model for; init = list of nodes that are activated at t=0
    
    t = 0
    highNum = 0
    
    #avg_weight = statistics.mean([e[2]['Weight'] for e in G.edges(data=True)])
    #max_weight = max([e[2]['Weight'] for e in G.edges(data=True)])
    multiplier = 0.1
    
    activation_times = {}
    for i in init:
        activation_times[i]=0
    
    while t<t_tot:
    
        curr_infectious = [n for n in activation_times if activation_times[n]==t]

        for n in curr_infectious:
            for m in G.neighbors(n):
                if m not in activation_times.keys():
                    multiplier = 0.1
                    if(G[n][m]['UniqueTeams'] > 1 ):
                        multiplier+= (0.05 * G[n][m]['UniqueTeams'])

                    if(G[n][m]['Nation1'] == G[n][m]['Nation2']):
                        multiplier+= 0.2
                        
                    p = G[n][m]['Weight']
                    if(p > 8):
                        multiplier+= 0.65
                        highNum+= 1
                    elif(p > 5):
                        multiplier+= 0.4
                        highNum+= 1
                    elif(p > 3):
                        multiplier+= 0.1
                    
                    # if p>random.uniform(0,1)*10:
                    #     activation_times[m] = t+1
                    if(multiplier >= 1):
                        #print(n, m)
                        multiplier = 0.99
                    check = math.floor( random.uniform(0, 1/(1-multiplier)))
                    # print(check, p)

                    if(check == 1):
                        activation_times[m] = t+1
                    
                        
                        
        t+=1

    return activation_times

### Create Graph from edge list.


In [13]:
edgesDf = pd.read_csv("edgesDataComp3.csv")
edgeCopy = edgesDf
Squads = edgeCopy['Squad']
uniqueTeams = []
for i in range(len(Squads)):
    uniqueTeams.append(len(set(Squads[i])))
edgeCopy['UniqueTeams'] = uniqueTeams
edgeCopy.rename(columns={'Source': 'source' ,'Target' : 'target', 'seasonsPlayed': 'Weight'}, inplace=True)

G = nx.from_pandas_edgelist(edgeCopy, edge_attr=['Season', 'Weight', 'Name1', 'Name2','Nation1','Nation2', 'UniqueTeams', 'Born1','Born2'])


c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Add Node Attributes to the graph from node list

In [27]:
nodes  = pd.read_csv("playerNodesData.csv")
nodes_attr = nodes.set_index('ID').to_dict(orient = 'index')
nx.set_node_attributes(G, nodes_attr)
for i in G.nodes():
    if G.nodes[i]['DOB'] == 'na Na':
        print('hi')
        print(i)
        G.nodes[i]['DOB'] = 1990
nameDict = pd.Series(nodes.Name.values, index=nodes.ID).to_dict()

hi
29d80748


In [14]:
degreeList = [(G.degree(i), i) for i in G.nodes()]
degreeList.sort(reverse=True)

Get total degree of node groups

In [16]:
barcaNeighbors = []
for m in G.neighbors('c1b4188c'):
    barcaNeighbors.append(m)

for m in G.neighbors('cfd65a29'):
    barcaNeighbors.append(m)

for m in G.neighbors('d70ce98e'):
    barcaNeighbors.append(m)

for m in G.neighbors('adfc9123'):
    barcaNeighbors.append(m)

for m in G.neighbors('7c0bda9b'):
    barcaNeighbors.append(m)


badBoyz = []
for m in G.neighbors('68517cfd'):
    badBoyz.append(m)

for m in G.neighbors('a6154613'):
    badBoyz.append(m)

for m in G.neighbors('69384e5d'):
    badBoyz.append(m)

for m in G.neighbors('561de8d0'):
    badBoyz.append(m)

for m in G.neighbors('5271c8f1'):
    badBoyz.append(m)

goodBoyz = []
for m in G.neighbors('29733c72'):
    goodBoyz.append(m)

for m in G.neighbors('c1b4188c'):
    goodBoyz.append(m)

for m in G.neighbors('a1d5bd30'):
    goodBoyz.append(m)

for m in G.neighbors('5e36a0dd'):
    goodBoyz.append(m)

for m in G.neighbors('945dea33'):
    goodBoyz.append(m)

print(len(set(barcaNeighbors)), len(set(badBoyz)), len(set(goodBoyz)))

259 637 507


Random ensemble ran information cascade 1000 times for 10 nodes each time

In [18]:
from random import sample
ensembleInflu = []
degreeAvg = []
i = 0
rounds = 1
for i in range(1000):
    random_nodes = sample(list(G.nodes()), 10)
    degsum = 0
    for j in random_nodes:
        degsum += G.degree(j)
    degreeAvg.append(degsum)
    result = len((information_cascade(G, rounds, random_nodes)))
    ensembleInflu.append(result)

print(np.mean(ensembleInflu))
print(np.std(ensembleInflu))

print(np.mean(degreeAvg))
print(np.std(degreeAvg))

338.516
61.9595331163817
1160.699
185.43470656541078


Run on bottom and top 10 degree groups

In [ ]:
top10Degs = degreeList[0:10]
bot10Degs = degreeList[9690:]

In [ ]:
ensembleTop10= []
ensemblBottom10= []
i = 0
rounds = 1
for i in range(100):
    print(i)
    top10Boys = len(information_cascade(G,rounds, top10Degs))
    bot10Boys = len(information_cascade(G, rounds, bot10Degs))
    ensembleTop10.append((top10Boys))
    ensemblBottom10.append((bot10Boys))

print(np.mean(ensembleTop10))
print(np.std(ensembleTop10))

print(np.mean(ensemblBottom10))
print(np.std(ensemblBottom10))

Run information cascade on the three groups of players choosen

In [25]:
ensembleBad= []
ensembleGood = []
ensembleBarca = []
i = 0
rounds = 1
for i in range(100):
    barcaGoats = len(information_cascade(G,rounds, ['c1b4188c','cfd65a29','d70ce98e','adfc9123','7c0bda9b']))
    goodGuys = len(information_cascade(G, rounds, ['29733c72', 'c1b4188c', 'a1d5bd30','5e36a0dd', '945dea33']))
    badGuys = len(information_cascade(G, rounds, ['68517cfd','a6154613','69384e5d','561de8d0','5271c8f1']))
    ensembleBad.append((badGuys))
    ensembleGood.append((goodGuys))
    ensembleBarca.append((barcaGoats))


160.17
9.743772370083365

133.2
8.34386001800126

97.87
5.583287562001441


In [ ]:
print(np.mean(ensembleBad))
print(np.std(ensembleBad))
print()
print(np.mean(ensembleGood))
print(np.std(ensembleGood))
print()
print(np.mean(ensembleBarca))
print(np.std(ensembleBarca))

Test age influence on various players

In [22]:
result1 = age_influence(G,1, ['000b3da6']) #high number of 1's and 2's
result2 = age_influence(G,1, ['561de8d0']) #Giggs
result3 = age_influence(G,1, ['1b013e33']) # G Nev
result4 = age_influence(G,1, ['4cde5509']) # Zlatan
result5 = age_influence(G,1, ['7c0bda9b']) # Puyol
result6 = age_influence(G,1, ['f4ec4fd0']) # Kevin Prince
result7 = age_influence(G,1, ['5bc43860']) # Brendan Aaronsson
result8 = age_influence(G,1, ['d70ce98e']) # GOAT

print(len(result1), len(result2), len(result3), len(result4), len(result5), len(result6), len(result7), len(result8))

43 78
48 21
23 17
89 62
22 23
85 102
0 0
45 45
42 23 15 63 17 86 7 59


Run age influence on groups of players

In [ ]:
ensembleBad= []
ensembleGood = []
ensembleBarca = []
i = 0
rounds = 1
for i in range(100):
    barcaGoats = len(age_influence(G,rounds, ['c1b4188c','cfd65a29','d70ce98e','adfc9123','7c0bda9b']))
    goodGuys = len(age_influence(G, rounds, ['29733c72', 'c1b4188c', 'a1d5bd30','5e36a0dd', '945dea33']))
    badGuys = len(age_influence(G, rounds, ['68517cfd','a6154613','69384e5d','561de8d0','5271c8f1']))
    ensembleBad.append((badGuys))
    ensembleGood.append((goodGuys))
    ensembleBarca.append((barcaGoats))


In [36]:
print(np.mean(ensembleBad))
print(np.std(ensembleBad))
print()
print(np.mean(ensembleGood))
print(np.std(ensembleGood))
print()
print(np.mean(ensembleBarca))
print(np.std(ensembleBarca))

160.76
9.539517807520461

145.61
7.8560740831537474

115.58
6.71294272283028


### Running age influence on just Ryan Giggs

In [26]:
ryanGiggs = age_influence(G, 1, ["561de8d0"]) #Giggs not Mendy561de8d0
print(len(ryanGiggs))

48 21
23


In [30]:
giggsInfluence = pd.DataFrame(ryanGiggs.items(), columns=['ID', 'Influenced'])
giggsInfluence['Name'] = giggsInfluence['ID'].map(nameDict)
giggsInfluence

,ID,Influenced,Name
0,561de8d0,0,Ryan-Giggs
1,1ae30132,1,Phil-Bardsley
2,3fcab3a8,1,Robin-van-Persie
3,4737cebe,1,Adnan-Januzaj
4,51a3caf1,1,Patrice-Evra
5,54629cf9,1,Gabriel-Heinze
6,5afafbfe,1,Shinji-Kagawa
7,636f1a28,1,Anders-Lindegaard
8,688135a1,1,Danny-Higginbotham
9,7106e4fe,1,Louis-Saha


In [31]:
giggsInfluence = giggsInfluence.merge(nodes, left_on="ID", right_on="ID", how='left')
giggsInfluence['Age'] = 2022 - giggsInfluence['DOB'].astype(int)
giggsInfluence = giggsInfluence[['ID', 'Name_x', 'CountryFixed', 'Age']]
borb = []
for i in giggsInfluence['ID']:
    borb.append(G.degree(i))
giggsInfluence['Degree'] = borb
giggsInfluence

,ID,Name_x,CountryFixed,Age,Degree
0,561de8d0,Ryan-Giggs,Wales,49,77
1,1ae30132,Phil-Bardsley,Scotland,37,226
2,3fcab3a8,Robin-van-Persie,Netherlands,39,157
3,4737cebe,Adnan-Januzaj,Belgium,27,206
4,51a3caf1,Patrice-Evra,France,41,212
5,54629cf9,Gabriel-Heinze,Argentina,44,127
6,5afafbfe,Shinji-Kagawa,Japan,33,183
7,636f1a28,Anders-Lindegaard,Denmark,38,129
8,688135a1,Danny-Higginbotham,Gibraltar,44,112
9,7106e4fe,Louis-Saha,France,44,162


Get top influencers

In [ ]:
influenceListVal = []
influenceName = []

for i in G.nodes():
    
    influencers = age_influence(G, 1, [i])
    influenceListVal.append(len(influencers) - 1)
    influenceName.append(i)
influeDf = pd.DataFrame()
influeDf['ID'] = influenceName
influeDf['InfluenceCount'] = influenceListVal
influeDf["Name"] = influeDf['ID'].map(nameDict)

Fun Degree Ratio excerise. Shows the comparssion between total players and unique players played with. 

In [34]:
degrees = [G.degree(n, weight="Weight") for n in G.nodes()]
degreesNonWeight = [G.degree(n) for n in G.nodes()]
nodesList = [n for n in G.nodes()]
degreeRatio = []
degreeNames = []
degreeSeasons = []
degreeComp = []
degreeSquad = []
numPlayers = []
numSeasonsPlayers = []
j = 0
for i in degrees:
    # if(i > 550):
    # degreeNames.append(G.nodes[nodesList[j]]["Name"] + " " + G.nodes[nodesList[j]]["DOB"] + " Weighted Num Seasons " + str(degrees[j]) + " Num Players " + str(degreesNonWeight[j]) + " Ratio (PlayedWithSeasons : TotalPlayedWith " + str(degrees[j] / degreesNonWeight[j])) 
    degreeNames.append(G.nodes[nodesList[j]]["Name"])
    degreeSeasons.append(G.nodes[nodesList[j]]["Season"])
    degreeComp.append(G.nodes[nodesList[j]]["Comp"])
    degreeSquad.append(G.nodes[nodesList[j]]["Squad"])
    degreeRatio.append(degreesNonWeight[j] / degrees[j])
    numSeasonsPlayers.append((degrees[j]))
    numPlayers.append(degreesNonWeight[j])

    j+=1

df2 = pd.DataFrame(degreeNames, columns=['Name'])
df2["Ratio"] = degreeRatio
df2["Total_Players"] = numPlayers
df2["Total_Seasons"] = numSeasonsPlayers
df2["Season"] = degreeSeasons
df2["Comp"] = degreeComp
df2["Squad"] = degreeSquad
df2["NewPlayers/Season"] = df2["Total_Players"] / df2["Season"]
df2["NewPlayers/Squad"] = df2["Total_Players"] / df2["Squad"]
df2["NewPlayers/Comp"] = df2["Total_Players"] / df2["Comp"]
df2.sort_values(["Ratio"])


,Name,Ratio,Total_Players,Total_Seasons,Season,Comp,Squad,NewPlayers/Season,NewPlayers/Squad,NewPlayers/Comp
3279,Ryan-Giggs,0.234756,77,328,22,1,1,3.500000,77.00,77.0
7300,Iker-Muniain,0.238227,86,361,14,1,1,6.142857,86.00,86.0
1395,Paul-Scholes,0.238255,71,298,19,1,1,3.736842,71.00,71.0
6091,Oscar-de-Marcos,0.245902,90,366,15,2,2,6.000000,45.00,45.0
4029,Carlos-Gurpegui,0.248908,57,229,14,1,1,4.071429,57.00,57.0
...,...,...,...,...,...,...,...,...,...,...
766,Dorian-Caddy,1.000000,63,63,4,2,4,15.750000,15.75,31.5
2431,Jean-Ambrose,1.000000,36,36,2,2,2,18.000000,18.00,18.0
2455,Fabio-Della-Giovanna,1.000000,87,87,3,2,3,29.000000,29.00,43.5
2032,Sergio-Segura-Garcia,1.000000,28,28,1,1,1,28.000000,28.00,28.0
